In [1]:
from PIL import Image
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict
import time
import numpy as np
from timm.utils import AverageMeter
import mycv


/home/jackfruit/a/duan90/anaconda3/envs/py311pt22/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_paths = sorted(mycv.dataset_paths['kodak'].rglob('*.png'))
print('Number of images:', len(img_paths))

Number of images: 24


In [3]:
def benchmark(codec, desc=''):
    tmp_path = Path('runs/tmp.bits')
    sanity_flag = True
    metrics = defaultdict(AverageMeter)

    for impath in tqdm(img_paths, ascii=True, ncols=96, desc=desc):
        # encode
        start_time = time.time()
        codec.encode_file(impath, tmp_path)
        encode_end = time.time()

        # decode
        im = codec.decode_file(tmp_path)
        decode_end = time.time()

        # sanity check
        original = np.array(Image.open(impath))
        if not np.array_equal(original, im):
            sanity_flag = False

        # metrics
        num_bits = tmp_path.stat().st_size * 8
        bpp = num_bits / float(original.shape[0] * original.shape[1])
        metrics['enc (s)'].update(encode_end - start_time)
        metrics['dec (s)'].update(decode_end - encode_end)
        metrics['bpp'].update(bpp)

        # cleanup
        tmp_path.unlink()

    if not sanity_flag:
        print('==== Warning: not lossless ====')

    metrics = {k: v.avg for k, v in metrics.items()}
    metrics['ratio'] = metrics['bpp'] / 24
    mycv.utils.print_dict_as_table(metrics)
    print()

## PNG

In [4]:
class PNGCodec():
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def encode_file(self, impath, tmp_path):
        im = np.array(Image.open(impath))
        Image.fromarray(im).save(tmp_path, format='png', **self.kwargs)

    def decode_file(self, tmp_path):
        im = np.array(Image.open(tmp_path))
        return im

benchmark(PNGCodec(), 'PNG')
benchmark(PNGCodec(optimize=True), 'PNG, optimize=True')

PNG: 100%|######################################################| 24/24 [00:03<00:00,  6.37it/s]


| enc (s) | dec (s) |  bpp   | ratio  |
| 0.1327  | 0.01246 | 13.51  | 0.5631 |



PNG, optimize=True: 100%|#######################################| 24/24 [00:05<00:00,  4.12it/s]

| enc (s) | dec (s) |  bpp   | ratio  |
| 0.2203  | 0.01066 |  13.4  | 0.5584 |



## WebP

In [5]:
class WebPCodec():
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def encode_file(self, impath, tmp_path):
        im = np.array(Image.open(impath))
        Image.fromarray(im).save(tmp_path, format='webp', **self.kwargs)

    def decode_file(self, tmp_path):
        im = np.array(Image.open(tmp_path))
        return im

benchmark(WebPCodec(lossless=True), 'WebP lossless')
benchmark(WebPCodec(lossless=True, quality=0), 'WebP lossless, quality=0')
benchmark(WebPCodec(lossless=True, quality=100), 'WebP lossless, quality=100')

WebP lossless: 100%|############################################| 24/24 [00:08<00:00,  2.98it/s]


| enc (s) | dec (s) |  bpp   | ratio  |
| 0.3127  | 0.01131 | 9.612  | 0.4005 |



WebP lossless, quality=0: 100%|#################################| 24/24 [00:04<00:00,  4.99it/s]


| enc (s) | dec (s) |  bpp   | ratio  |
| 0.1781  | 0.01084 |  9.7   | 0.4042 |



WebP lossless, quality=100: 100%|###############################| 24/24 [00:26<00:00,  1.09s/it]

| enc (s) | dec (s) |  bpp   | ratio  |
|  1.064  | 0.01154 | 9.569  | 0.3987 |



## AV1 Image File (AVIF)

In [6]:
import pillow_avif

class AVIFCodec():
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def encode_file(self, impath, tmp_path):
        im = np.array(Image.open(impath))
        Image.fromarray(im).save(tmp_path, format='avif', **self.kwargs)

    def decode_file(self, tmp_path):
        im = np.array(Image.open(tmp_path))
        return im

benchmark(AVIFCodec(quality=100), 'AVIF')

AVIF: 100%|#####################################################| 24/24 [00:05<00:00,  4.37it/s]

==== Warning: not lossless ====
| enc (s) | dec (s) |  bpp   | ratio  |
| 0.1917  | 0.02436 | 5.816  | 0.2423 |



## High Efficiency Image File (HEIF)

In [7]:
from pillow_heif import register_heif_opener
register_heif_opener()

class HEIFCodec():
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def encode_file(self, impath, tmp_path):
        im = np.array(Image.open(impath))
        Image.fromarray(im).save(tmp_path, format='heif', quality=-1, chroma=444, matrix_coefficients=0, **self.kwargs)

    def decode_file(self, tmp_path):
        im = np.array(Image.open(tmp_path))
        return im

benchmark(HEIFCodec(), 'HEIF')

HEIF: 100%|#####################################################| 24/24 [00:13<00:00,  1.83it/s]

| enc (s) | dec (s) |  bpp   | ratio  |
| 0.4113  | 0.1105  | 14.13  | 0.5889 |



## JPEG XL

In [8]:
from jxlpy import JXLImagePlugin

class JPEGXLCodec():
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def encode_file(self, impath, tmp_path):
        im = np.array(Image.open(impath))
        Image.fromarray(im).save(tmp_path, format='jxl', lossless=True, **self.kwargs)

    def decode_file(self, tmp_path):
        im = np.array(Image.open(tmp_path))
        return im

benchmark(JPEGXLCodec(), 'JPEG-XL')
benchmark(JPEGXLCodec(effort=3), 'JPEG-XL, effort=3')
benchmark(JPEGXLCodec(effort=6), 'JPEG-XL, effort=6')
benchmark(JPEGXLCodec(effort=9), 'JPEG-XL, effort=9')

JPEG-XL: 100%|##################################################| 24/24 [00:21<00:00,  1.10it/s]


| enc (s) | dec (s) |  bpp   | ratio  |
| 0.7813  | 0.1123  | 8.706  | 0.3628 |



JPEG-XL, effort=3: 100%|########################################| 24/24 [00:03<00:00,  6.93it/s]


| enc (s) | dec (s) |  bpp   | ratio  |
| 0.0771  | 0.05554 |  9.44  | 0.3933 |



JPEG-XL, effort=6: 100%|########################################| 24/24 [00:16<00:00,  1.49it/s]


| enc (s) | dec (s) |  bpp   | ratio  |
| 0.5534  | 0.1075  | 8.828  | 0.3678 |



JPEG-XL, effort=9: 100%|########################################| 24/24 [01:26<00:00,  3.61s/it]

| enc (s) | dec (s) |  bpp   | ratio  |
|  3.494  | 0.1042  | 8.665  | 0.3611 |

